# 

In [ ]:
import pyemma
pyemma.__version__
import os
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import glob
import MDAnalysis as mda
import MDAnalysis.transformations
from tqdm import tqdm_notebook as tqdm
import pyemma.coordinates as coor
import warnings

# Conduct PCA

In [ ]:
curre_dir=%pwd
all_xtc=sorted(glob.glob(f"../gerbil_main/candi*/*/amd_noPBC.xtc"))
top=glob.glob("../gerbil_main/input/em.gro")[0]
print(top,len(all_xtc))

In [ ]:
traj_c_alpha=[]
ref = mda.Universe(top)
ref_c_alpha = ref.select_atoms("name CA")
for traj in tqdm(all_xtc):
    u = mda.Universe(top, traj)
    c_alpha = u.select_atoms("name CA")
    transform = mda.transformations.fit_rot_trans(c_alpha, ref_c_alpha ,weights="mass")
    u.trajectory.add_transformations(transform)
    for ts in (u.trajectory):
        traj_c_alpha.append(list(itertools.chain.from_iterable(c_alpha.positions)))
print(np.shape(traj_c_alpha))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(traj_c_alpha)
init_pca = pca.transform(traj_c_alpha)

In [ ]:
plt.plot([0] + list(np.cumsum(pca.explained_variance_ratio_))[:10], linewidth=0, marker='o', markersize=3, color='black')
print(pca.explained_variance_ratio_[:10])

# 射影

In [ ]:
all_xtc=sorted(glob.glob("./candi*/cluser_noPBC.xtc"))
print(len(all_xtc))

In [ ]:
!./trajtop.sh 200

In [ ]:
for traj in tqdm(all_xtc):
        place=traj.replace("cluser_noPBC.xtc","")
        u = mda.Universe(traj.replace("cluser_noPBC.xtc","em_nowater.gro"),traj)
        c_alpha = u.select_atoms("name CA")
        transform = mda.transformations.fit_rot_trans(c_alpha, ref_c_alpha ,weights="mass")
        u.trajectory.add_transformations(transform)
        traj_c_alpha=[]
        for ts in u.trajectory:
            traj_c_alpha.append(list(itertools.chain.from_iterable(c_alpha.positions)))
        pca_result=pca.transform(traj_c_alpha)
        np.savetxt(f"{place}RC.txt",np.array([pca_result[:,0],pca_result[:,1]]).T)    

In [ ]:
all_reader=[]
for index,name in enumerate(sorted(glob.glob(f"./candi_*/RC.txt"))):
    all_reader.append(np.loadtxt(name))
all_XYZ=all_reader
all_X=np.vstack(all_XYZ)[:,0]
all_Y=np.vstack(all_XYZ)[:,1]
print(np.shape(all_X),np.shape(all_Y))

  # 結晶構造の射影

In [ ]:
def conduct_pca(name):
    ref = mda.Universe("../gerbil_main/input/em.gro")
    ref_c_alpha = ref.select_atoms("name CA")
    u = mda.Universe(name,name)
    c_alpha = u.select_atoms("name CA")
    transform = mda.transformations.fit_rot_trans(c_alpha, ref_c_alpha ,weights="mass")
    u.trajectory.add_transformations(transform)
    traj_c_alpha=[]
    for ts in (u.trajectory):
            traj_c_alpha.append(list(itertools.chain.from_iterable(c_alpha.positions)))
    pca_result=pca.transform(traj_c_alpha)  
    return [pca_result[:,0],pca_result[:,1]]

In [ ]:
op="../gerbil_main/input/em_nowater.gro"
cl="../input/closed_nowater.gro"
op_pca=(conduct_pca(op))
cl_pca=(conduct_pca(cl))
np.savetxt("./OP_RC.txt",np.array(op_pca).T)
np.savetxt("./CL_RC.txt",np.array(cl_pca).T)